In [111]:
import json
import matplotlib.pyplot as plt
import numpy as np
import cv2
from joblib import Parallel, delayed

In [112]:
def load_json(group,student):
    with open(f'json-cv/{group}-{student}.json') as f:
        return json.load(f)

In [81]:
data = []
bb = []
first_group = 9
last_group = 9
for group in range(first_group,last_group+1):
    for student in range(1,3+1):
        data.append(load_json(group,student))
        for frame in range(1,3+1):
            bb.append(load_json(group,str(student)+"-"+str(frame)))
        

[[265, 131, 606, 606]]
[[1317, 485, 606, 606], [568, 132, 596, 596]]
[[866, 106, 628, 628], [292, 272, 608, 608]]
[[843, 143, 610, 610], [196, 318, 600, 600]]
[[1136, 128, 616, 616], [496, 318, 608, 608]]
[[817, 327, 610, 610], [287, 139, 686, 686]]
[[506, 484, 616, 616], [1187, 199, 602, 602]]
[[790, 474, 620, 620], [66, 132, 604, 604]]
[[1085, 473, 606, 606], [367, 101, 610, 610]]


In [42]:
list = [[1,2,3],[4,5,3],[7,8,9],[1,2,3],[4,5,3],[7,8,9]]
array = np.array(list)
print(array.shape)

# Use boolean indexing to keep rows where the last element is 3
sorted_array = array[array[:,2] == 3, :]

print(sorted_array)


(6, 3)
[[1 2 3]
 [4 5 3]
 [1 2 3]
 [4 5 3]]


In [79]:
current_bb = bb[1]
print(len(bb))
print(current_bb)

9
[[1317, 485, 606, 606], [568, 132, 596, 596]]


In [121]:
group = 9
for student in range(1,3+1):
    for frame in range(1,3+1):
        print(frame)
        dict_data = {}
        current_data = data[student-1]
        current_bb = bb[(student-1)*3+frame-1]

        cell = np.array(current_data[f'{group}_{student}_frame_{frame}']["cells"])

        dict_data["bb"]=current_bb
        dict_data["valid_bb"] = []
        dict_data["cell"] = []
        
        for id in range(len(current_bb)):
            dict_data["valid_bb"].append(id)
            dict_data["cell"].append([])

        cells_id = np.unique(cell[:,2])
        print(cells_id)

        for test_id in cells_id:
            box_id_list = []    
            cell_current_bb = np.where(cell[:,2]==test_id)[0]
            for i in range(1,10):
                for bb_id, bb_el in enumerate(current_bb):
                    x_min = bb_el[0]
                    y_min = bb_el[1]
                    x_max = bb_el[0] + bb_el[2]
                    y_max = bb_el[1] + bb_el[3]
                    if 5*i < len(cell_current_bb) and cell[cell_current_bb[5*i]][0] > x_min and cell[cell_current_bb[5*i]][1] > y_min and cell[cell_current_bb[5*i]][0] < x_max and cell[cell_current_bb[5*i]][1] < y_max:
                        box_id_list.append(bb_id)
            # box_id contain the value the more represented in the list
            if len(box_id_list) > 0:
                box_id = max(set(box_id_list), key = box_id_list.count)
                dict_data["valid_bb"][box_id] = 1
                dict_data["cell"][box_id] = cell[cell[:,2] == test_id, :2].tolist()
            else:
                print("tested id: ",test_id)
                dict_data["valid_bb"][id] = 0

        file_path = f'generated/test{group}-{student}-{frame}.json'
        with open(file_path, 'w') as json_file:
            json.dump(dict_data, json_file)
        #print(dict_data)





1
[0 1]
tested id:  1
2
[0 1]
3
[0 2]
1
[1 2]
2
[1 2]
3
[2 3]
1
[1 2]
2
[2 3]
3
[2 3]
